In [ ]:
import sys
import xlsxwriter
import pandas as pd
import joblib
import numpy
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import pandas as pd
import numpy as np
import random
import gc
import skimage, os
import torch_geometric.nn as geo
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import normalize
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from collections import defaultdict as ddict
from torch.utils.data import DataLoader
#from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from tensorflow.keras.utils import to_categorical
from fast_pytorch_kmeans import KMeans
from torchsummary import summary
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import Sequential, GCNConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool
from torch import tensor
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from keras.utils import normalize
from skimage.color import rgb2gray
from torch.nn import Parameter
from torch.nn.init import xavier_uniform_,xavier_normal_,kaiming_uniform_,kaiming_normal_

In [ ]:
##Data Loading

Image_directory='C:/Users/modak/lung cancer/Noduledetection/data2/'
No_nodule=os.listdir(Image_directory+'train_patches_nonodule/')
Nodule=os.listdir(Image_directory+'train_patches_nodule/')
test_No_nodule=os.listdir(Image_directory+'test_patches_nonodule/')
test_Nodule=os.listdir(Image_directory+'test_patches_nodule/')
dataset=[]
label=[]
edge_index=[]
graphs=[]
data = ddict(list)
for i, image_name in enumerate(No_nodule):

    if(image_name.split('.')[1]=='png'):
        image=cv2.imread(Image_directory+'train_patches_nonodule/'+image_name)
        image=Image.fromarray(image, 'RGB')
        edges=np.loadtxt(Image_directory+'train_edges_nonodule/'+image_name.split('.')[0]+'edges.txt',dtype=int)
        features=np.loadtxt(Image_directory+'train_features_nonodule/'+image_name.split('.')[0]+'feat.txt',dtype=int)
        locations=np.loadtxt(Image_directory+'train_location_nonodule/'+image_name.split('.')[0]+'loc.txt',dtype=int)
        dataset.append(np.array(image))
        edge_index.append(np.array(edges))
        data['train'].append({'image':np.array(image),'features':np.mean(np.array(features),axis=1), 'edge_index':np.array(edges.T), 'locations' :np.array(locations), 'label':0})
        

for i, image_name in enumerate(Nodule):
    if(image_name.split('.')[1]=='png'):
        image=cv2.imread(Image_directory+'train_patches_nodule/'+image_name)
        image=Image.fromarray(image, 'RGB')
        edges=np.loadtxt(Image_directory+'train_edges_nodule/'+image_name.split('.')[0]+'edges.txt',dtype=int)
        features=np.loadtxt(Image_directory+'train_features_nodule/'+image_name.split('.')[0]+'feat.txt',dtype=int)
        locations=np.loadtxt(Image_directory+'train_location_nodule/'+image_name.split('.')[0]+'loc.txt',dtype=int)
        dataset.append(np.array(image))
        edge_index.append(np.array(edges))
        data['train'].append({'image':np.array(image),'features':np.mean(np.array(features),axis=1), 'edge_index':np.array(edges.T), 'locations' :np.array(locations), 'label':1})
        
        
for i, image_name in enumerate(test_No_nodule):

    if(image_name.split('.')[1]=='png'):
        image=cv2.imread(Image_directory+'test_patches_nonodule/'+image_name)
        image=Image.fromarray(image, 'RGB')
        edges=np.loadtxt(Image_directory+'test_edges_nonodule/'+image_name.split('.')[0]+'edges.txt',dtype=int)
        features=np.loadtxt(Image_directory+'test_features_nonodule/'+image_name.split('.')[0]+'feat.txt',dtype=int)
        locations=np.loadtxt(Image_directory+'test_location_nonodule/'+image_name.split('.')[0]+'loc.txt',dtype=int)
        #image=image.resize((256,256))
        dataset.append(np.array(image))
        edge_index.append(np.array(edges))
        data['test'].append({'image':np.array(image),'features':np.mean(np.array(features),axis=1), 'edge_index':np.array(edges.T), 'locations' :np.array(locations), 'label':0})
        

for i, image_name in enumerate(test_Nodule):
    if(image_name.split('.')[1]=='png'):
        image=cv2.imread(Image_directory+'test_patches_nodule/'+image_name)
        image=Image.fromarray(image, 'RGB')
        edges=np.loadtxt(Image_directory+'test_edges_nodule/'+image_name.split('.')[0]+'edges.txt',dtype=int)
        features=np.loadtxt(Image_directory+'test_features_nodule/'+image_name.split('.')[0]+'feat.txt',dtype=int)
        locations=np.loadtxt(Image_directory+'test_location_nodule/'+image_name.split('.')[0]+'loc.txt',dtype=int)
        dataset.append(np.array(image))
        edge_index.append(np.array(edges))
        data['test'].append({'image':np.array(image),'features':np.mean(np.array(features),axis=1), 'edge_index':np.array(edges.T), 'locations' :np.array(locations), 'label':1})
        



In [ ]:
len(data['train'])

In [ ]:
len(data['test'])

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.set_rng_state(torch.cuda.get_rng_state())
    torch.backends.cudnn.deterministic = True
else:
    device = torch.device('cpu')

In [ ]:
def dataloader(split):
    trainloader=DataLoader(data[split],
                batch_size=256,
                shuffle=True,# if split=='train' else False,
                num_workers=0,
                drop_last=True 
                
                )
    return trainloader


In [ ]:
def get_param(shape):
    param = Parameter(torch.Tensor(*shape));
    #kaiming_uniform_(param.data, a=0, mode='fan_out', nonlinearity='leaky_relu')
    
    kaiming_uniform_(param.data)
    return param

In [ ]:
## NSPN Deployment
from Conv_new import NSPN
class superpixelnetwork(torch.nn.Module):
    def __init__(self, input_features,hidden_channels):
        super(superpixelnetwork, self).__init__()
        torch.manual_seed(12345)

        self.numnodes=256
        self.input_features=input_features
        self.hidden_channels=hidden_channels
        self.bn0 = torch.nn.BatchNorm2d(3)
        self.bn1 = torch.nn.BatchNorm2d(3)

        self.conv1 = NSPN(self.input_features, int(self.hidden_channels/2),self.numnodes)
        self.conv2 = NSPN(int(self.hidden_channels/2),self.hidden_channels,self.numnodes)
        self.weighte= get_param((3,32, 32))
        self.weightg= get_param((3,16,16))
        
        self.pool1 = nn.MaxPool2d(4,4)
        self.pool0 = nn.MaxPool2d(8,8)

        self.fc1 = torch.nn.Linear(7936,128 ,bias=True) 
        self.fc2 = torch.nn.Linear(128,1 ,bias=True) 


    def forward(self, x,image,locs, edge_index):
        
        image=image.view(-1,3,128,128)/256
        
        
        x=x/256
        x=x.view(-1,x.shape[1],1)
        
        
        for i in range(0,x.shape[0]):
            
            
            y = self.conv1.forward(x=x[i], edge_index=edge_index[i].type(torch.int64))
           
            y=self.conv2.forward(x=y, edge_index=edge_index[i].type(torch.int64))

            y=F.relu(y)
            

            if i==0:
                z1=y
                
            else:
                
                z1=torch.cat((z1,y),0)
                
        
        
        z1=z1.view(-1,1,self.numnodes,self.hidden_channels)

        z1=z1.view(-1,z1.shape[1]*z1.shape[2]*z1.shape[3])
        
        
        
        z2=self.pool1(image)
        
        z2=torch.matmul(z2,self.weighte)
        z2=self.bn1(z2)
        z2=F.relu(z2)
        
        z0=self.pool0(image)
        
        z0=torch.matmul(z0,self.weightg)
        z0=self.bn0(z0)
        z0=F.relu(z0)
        
        
        
        z2=z2.view(-1,z2.shape[1]*z2.shape[2]*z2.shape[3])
        z0=z0.view(-1,z0.shape[1]*z0.shape[2]*z0.shape[3])
        
        z=self.fc1(torch.cat((z0,z1,z2),1))
        
        
  
        z=self.fc2(z)

        z = torch.sigmoid(z)
        #print(z)
        return z

model = superpixelnetwork(input_features=1,hidden_channels=16)
model.double()

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
model.to(device)
print(model)
print(pytorch_total_params)

In [ ]:
## Model Training 

num_epochs = 100

optimizer=torch.optim.Adam(model.parameters(),lr=0.00001, weight_decay=0.001)
loss_vec_train=[]
loss_vec_test=[]
epc=[]
acc=[]
acc2=[]
losterm=nn.BCELoss()
for epoch in range(num_epochs):
    model.train()
    losses = []
    correction=[]
    correction2=[]
    losses2=[]
    label_prime=[]
    predicted_prime=[]
    roc_test=[]  
    for batch in dataloader('train'):
        
        optimizer.zero_grad()
        output = model.forward((batch['features']).to(device),(batch['image'].double()).to(device),(batch['locations'].double()).to(device), (batch['edge_index']).to(device))
        

        label=torch.tensor(batch['label'],dtype=torch.double,requires_grad=False)
        #print(label.shape)
        output=output.view(-1)

        loss = F.binary_cross_entropy(output, label.to(device))
        loss.backward()
        optimizer.step()
        losses.append((loss.cpu()).detach().numpy())
         
        pred = torch.round(output)


        correct = int((pred.cpu() == batch['label']).sum())
        correction.append(correct)
        
        
    correction=np.array(correction)
    accuracy=np.sum(correction)/len(data['train'])
    losses=np.mean(losses)
    
    model.eval()
    for batch2 in dataloader('test'):
        
        evaluate=model.forward((batch2['features']).to(device),(batch2['image'].double()).to(device),(batch2['locations'].double()).to(device), (batch2['edge_index']).to(device))
        
        label2=torch.tensor(batch2['label'],dtype=torch.double,requires_grad=False)
        evaluate=evaluate.view(-1)
        
        loss2 = F.binary_cross_entropy(evaluate, label2.to(device))
        label_prime.append((label2.cpu()).detach().numpy())
        predicted_prime.append((evaluate.cpu()).detach().numpy())
        pred2 = torch.round(evaluate)
     
        correct2 = int((pred2.cpu() == batch2['label']).sum())
        correction2.append(correct2)
        losses2.append((loss2.cpu()).detach().numpy())
    predicted_prime=np.array(predicted_prime)
    label_prime=np.array(label_prime)
    predicted_prime=np.array(np.reshape(predicted_prime,(predicted_prime.shape[0]*predicted_prime.shape[1],-1)))
    
    label_prime=np.array(np.reshape(label_prime,(label_prime.shape[0]*label_prime.shape[1],-1)))
    if epoch==49:
        lr_fpr, lr_tpr, _ =roc_curve(label_prime[:, 0], predicted_prime[:, 0])

    losses2=np.mean(losses2)
    correction2=np.array(correction2)
    accuracy2=np.sum(correction2)/len(data['test'])
    print('train accuracy:',round(accuracy,4),'test accuracy:',round(accuracy2,4),'train loss:',round(losses,4),'test loss:', round(losses2,4))
    loss_vec_train.append(losses)
    loss_vec_test.append(losses2)
    epc.append(epoch)
    acc.append(accuracy)
    acc2.append(accuracy2)

loss_vec_train=np.array(loss_vec_train)
loss_vec_test=np.array(loss_vec_test)
acc=np.array(acc)
acc2=np.array(acc2)
epc=np.array(epc)


In [ ]:
torch.save(model.state_dict(), 'C:/Users/modak/lung cancer/Noduledetection/model.pt')